In [1]:
import pandas as pd
import jieba
import jieba.analyse
jieba.set_dictionary('dict.idkrsi.txt')     # 改預設字典
jieba.analyse.set_stop_words("stopword.goatwang.kang.txt") #指定stopwords字典

In [2]:
content = "1,2,3,八仙樂園,我好好!\n你在幹嘛?\n哈哈哈.\n,4,5,6,北市八仙水上樂園昨晚發生粉塵爆炸，新北市衛生局統計到目前為止，由救護車送醫再加上自行送醫"
seg_list = jieba.cut(str(content), cut_all=False)   # jieba割成一個list
print(" ".join(seg_list))                           # list 每個elements用空格分開

Building prefix dict from /home/barry/projects/stock/dict.idkrsi.txt ...
Loading model from cache /tmp/jieba.ufc1c7fa02eb193cee2bb78b34d63c017.cache
Loading model cost 0.804 seconds.
Prefix dict has been built succesfully.


1 , 2 , 3 , 八仙 樂園 , 我 好好 ! 
 你 在 幹嘛 ? 
 哈哈 哈 . 
 , 4 , 5 , 6 , 北市 八仙水 上 樂園 昨晚 發生 粉塵 爆炸 ， 新 北市 衛生局 統計 到 目前 為止 ， 由 救護車 送醫 再 加上 自行 送醫


In [3]:
news = pd.read_excel('https://raw.githubusercontent.com/ywchiu/pytextmining/master/data/20150628news.xlsx')
news.head(2)

,Unnamed: 0,category,description,pubdate,title
0,0,社會/生活,新北市八仙水上樂園昨晚發生粉塵爆炸，新北市衛生局統計到目前為止，由救護車送醫再加上自行送醫的...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙塵爆 五相關人依公共危險重傷害法辦
1,1,社會/生活,新北市八仙樂園昨天晚上(6/27)舉辦活動，過程中噴灑大量玉米粉而引發粉塵爆炸，根據最新統計...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙樂園意外 病患持續增加中


In [4]:
news = news.drop(columns=['Unnamed: 0'])   # 刪除某columns
#news = news.drop('Unnamed: 0', axis = 1)    # 刪除某columns, axis=1
#news = news.drop([0], axis = 0)             # 刪除row axis=0     根據index 0 
#news = news.set_index("pubdate")            # 用某欄位當index
news = news.reset_index(drop=True)           # index變成 0~N
news.head()

,category,description,pubdate,title
0,社會/生活,新北市八仙水上樂園昨晚發生粉塵爆炸，新北市衛生局統計到目前為止，由救護車送醫再加上自行送醫的...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙塵爆 五相關人依公共危險重傷害法辦
1,社會/生活,新北市八仙樂園昨天晚上(6/27)舉辦活動，過程中噴灑大量玉米粉而引發粉塵爆炸，根據最新統計...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙樂園意外 病患持續增加中
2,財經/要聞,希臘債務違約限期逼近，資金持續外流。路透社引述三間銀行的消息指出，希臘國內有3分之1的自動櫃...,"Sun, 28 Jun 2015 07:40:00 +0800",希臘國內三分一自動櫃員機現金短缺
3,社會/生活,台鐵「新基隆車站」第一階段工程本月底完成，台鐵表示，明天（29號）啟用後，由於南站周邊道路尚...,"Sun, 28 Jun 2015 07:40:00 +0800",台鐵新基隆車站29日正式啟用
4,財經/要聞,《中國時報》\n●樂園變煉獄 派對驚爆 逾300人遭火紋身\n八仙樂園昨晚舉辦「COLOR ...,"Sun, 28 Jun 2015 07:38:17 +0800",6月28日各報頭版要聞


In [5]:
news.loc[0:5,'title']          # 選取 [ index , column name]

0    八仙塵爆  五相關人依公共危險重傷害法辦
1          八仙樂園意外 病患持續增加中
2        希臘國內三分一自動櫃員機現金短缺
3          台鐵新基隆車站29日正式啟用
4             6月28日各報頭版要聞
5       基改不算真食物  法國人知情就不買
Name: title, dtype: object

In [6]:
new2 = news[['title','description']].head(2).copy()     #選指定columns 成新dataframe  # copy()才會產生新的
new2

,title,description
0,八仙塵爆 五相關人依公共危險重傷害法辦,新北市八仙水上樂園昨晚發生粉塵爆炸，新北市衛生局統計到目前為止，由救護車送醫再加上自行送醫的...
1,八仙樂園意外 病患持續增加中,新北市八仙樂園昨天晚上(6/27)舉辦活動，過程中噴灑大量玉米粉而引發粉塵爆炸，根據最新統計...


In [7]:
df_td =  news[['title','description']]
np_td = df_td.to_numpy()    # dataframe轉指定columns成array 0.23以前用 df.values
type(np_td)

numpy.ndarray

In [8]:
df_td = df_td.astype(str)               # dataframe 全部轉 string

In [9]:
np_td = np_td.astype(str)                   # numpy 全部轉 string

In [10]:
# description全部切詞

corpus = []  # array
for rec in news.iterrows():   
    seg_list = str( rec[1]['description'] ).replace('\n','').replace(',',' ')
    seg_list = jieba.cut(seg_list, cut_all=False)
    corpus.append(' '.join(seg_list))

news["description2"]=corpus  
news.head(2)

,category,description,pubdate,title,description2
0,社會/生活,新北市八仙水上樂園昨晚發生粉塵爆炸，新北市衛生局統計到目前為止，由救護車送醫再加上自行送醫的...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙塵爆 五相關人依公共危險重傷害法辦,新 北市 八仙水 上 樂園 昨晚 發生 粉塵 爆炸 ， 新 北市 衛生局 統計 到 目前 為...
1,社會/生活,新北市八仙樂園昨天晚上(6/27)舉辦活動，過程中噴灑大量玉米粉而引發粉塵爆炸，根據最新統計...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙樂園意外 病患持續增加中,新 北市 八仙 樂園 昨天 晚上 ( 6 / 27 ) 舉辦 活動 ， 過程 中 噴灑 大量...


In [11]:
corpus = []
for rec in news.iterrows():
    seg_list = str( rec[1]['title'] ).replace('\n','').replace(',',' ')
    seg_list = jieba.cut(seg_list, cut_all=False)
    corpus.append(' '.join(seg_list))
    
news["title2"]=corpus
news.head(2)

,category,description,pubdate,title,description2,title2
0,社會/生活,新北市八仙水上樂園昨晚發生粉塵爆炸，新北市衛生局統計到目前為止，由救護車送醫再加上自行送醫的...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙塵爆 五相關人依公共危險重傷害法辦,新 北市 八仙水 上 樂園 昨晚 發生 粉塵 爆炸 ， 新 北市 衛生局 統計 到 目前 為...,八仙 塵爆 五 相關 人依 公共 危險 重 傷害 法辦
1,社會/生活,新北市八仙樂園昨天晚上(6/27)舉辦活動，過程中噴灑大量玉米粉而引發粉塵爆炸，根據最新統計...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙樂園意外 病患持續增加中,新 北市 八仙 樂園 昨天 晚上 ( 6 / 27 ) 舉辦 活動 ， 過程 中 噴灑 大量...,八仙 樂園 意外 病患 持續 增加 中


In [12]:
content = "1.原文連結(必須檢附)：\nhttp://magazine.chinatimes.com/"
content.replace('\n',',') # \n 換成 ,

'1.原文連結(必須檢附)：,http://magazine.chinatimes.com/'

In [13]:
# content = open("bda2019_dataset/forum2.txt", "rb").read()               # 切整個txt
# seg_list = jieba.cut(str( content ), cut_all=False)                            # 精確模式
# print(" ".join(seg_list),file=open("bda2019_dataset/forum3.txt", "a"))  # print存成txt

In [14]:
news2 = news[ news['description'].str.contains('八仙')]   # df 某欄位 string contains "八仙"
news2.head()

,category,description,pubdate,title,description2,title2
0,社會/生活,新北市八仙水上樂園昨晚發生粉塵爆炸，新北市衛生局統計到目前為止，由救護車送醫再加上自行送醫的...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙塵爆 五相關人依公共危險重傷害法辦,新 北市 八仙水 上 樂園 昨晚 發生 粉塵 爆炸 ， 新 北市 衛生局 統計 到 目前 為...,八仙 塵爆 五 相關 人依 公共 危險 重 傷害 法辦
1,社會/生活,新北市八仙樂園昨天晚上(6/27)舉辦活動，過程中噴灑大量玉米粉而引發粉塵爆炸，根據最新統計...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙樂園意外 病患持續增加中,新 北市 八仙 樂園 昨天 晚上 ( 6 / 27 ) 舉辦 活動 ， 過程 中 噴灑 大量...,八仙 樂園 意外 病患 持續 增加 中
4,財經/要聞,《中國時報》\n●樂園變煉獄 派對驚爆 逾300人遭火紋身\n八仙樂園昨晚舉辦「COLOR ...,"Sun, 28 Jun 2015 07:38:17 +0800",6月28日各報頭版要聞,《 中國時報 》 ● 樂園 變 煉獄 派對 驚爆 逾 300 人 遭火 紋身 八仙 ...,6 月 28 日 各報 頭版 要聞
6,財經/要聞,傷者不斷被送進醫院，連計程車都加入，載送病患行列。誇張的是，有人是直接坐在救生圈上送醫，年輕...,"Sun, 28 Jun 2015 07:28:20 +0800",八仙樂園舞台大火 逾400人輕重傷,傷者 不斷 被 送進 醫院 ， 連 計程車 都 加入 ， 載送 病患 行列 。 誇張 的 是...,八仙 樂園 舞台 大火 逾 400 人 輕重傷
11,社會/生活,新北市八里的八仙樂園昨天晚間發生粉塵爆炸意外，造成數百人燒燙傷。行政院長毛治國今天（28號）...,"Sun, 28 Jun 2015 07:10:00 +0800",八仙樂園意外 毛揆取消視察臺東行程,新 北市 八里 的 八仙 樂園 昨天 晚間 發生 粉塵 爆炸 意外 ， 造成 數百人 燒燙傷...,八仙 樂園 意外 毛揆 取消 視察 臺東 行程


In [15]:
news['Date'] = pd.to_datetime(news['pubdate']) #轉時間格式成datetime
news.head(2)

,category,description,pubdate,title,description2,title2,Date
0,社會/生活,新北市八仙水上樂園昨晚發生粉塵爆炸，新北市衛生局統計到目前為止，由救護車送醫再加上自行送醫的...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙塵爆 五相關人依公共危險重傷害法辦,新 北市 八仙水 上 樂園 昨晚 發生 粉塵 爆炸 ， 新 北市 衛生局 統計 到 目前 為...,八仙 塵爆 五 相關 人依 公共 危險 重 傷害 法辦,2015-06-28 07:40:00+08:00
1,社會/生活,新北市八仙樂園昨天晚上(6/27)舉辦活動，過程中噴灑大量玉米粉而引發粉塵爆炸，根據最新統計...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙樂園意外 病患持續增加中,新 北市 八仙 樂園 昨天 晚上 ( 6 / 27 ) 舉辦 活動 ， 過程 中 噴灑 大量...,八仙 樂園 意外 病患 持續 增加 中,2015-06-28 07:40:00+08:00


In [16]:
news['DateShift'] = news['Date'].shift(periods=1)  # 移動一格 可做兩row 上下比較
news.head(10)

,category,description,pubdate,title,description2,title2,Date,DateShift
0,社會/生活,新北市八仙水上樂園昨晚發生粉塵爆炸，新北市衛生局統計到目前為止，由救護車送醫再加上自行送醫的...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙塵爆 五相關人依公共危險重傷害法辦,新 北市 八仙水 上 樂園 昨晚 發生 粉塵 爆炸 ， 新 北市 衛生局 統計 到 目前 為...,八仙 塵爆 五 相關 人依 公共 危險 重 傷害 法辦,2015-06-28 07:40:00+08:00,NaT
1,社會/生活,新北市八仙樂園昨天晚上(6/27)舉辦活動，過程中噴灑大量玉米粉而引發粉塵爆炸，根據最新統計...,"Sun, 28 Jun 2015 07:40:00 +0800",八仙樂園意外 病患持續增加中,新 北市 八仙 樂園 昨天 晚上 ( 6 / 27 ) 舉辦 活動 ， 過程 中 噴灑 大量...,八仙 樂園 意外 病患 持續 增加 中,2015-06-28 07:40:00+08:00,2015-06-28 07:40:00+08:00
2,財經/要聞,希臘債務違約限期逼近，資金持續外流。路透社引述三間銀行的消息指出，希臘國內有3分之1的自動櫃...,"Sun, 28 Jun 2015 07:40:00 +0800",希臘國內三分一自動櫃員機現金短缺,希臘 債務 違約 限期 逼近 ， 資金 持續 外流 。 路透社 引述 三間 銀行 的 消息 ...,希臘 國內 三分 一 自動 櫃員機 現金 短缺,2015-06-28 07:40:00+08:00,2015-06-28 07:40:00+08:00
3,社會/生活,台鐵「新基隆車站」第一階段工程本月底完成，台鐵表示，明天（29號）啟用後，由於南站周邊道路尚...,"Sun, 28 Jun 2015 07:40:00 +0800",台鐵新基隆車站29日正式啟用,台鐵 「 新 基隆 車站 」 第一 階段 工程 本月底 完成 ， 台鐵 表示 ， 明天 （ ...,台鐵 新 基隆 車站 29 日 正式 啟用,2015-06-28 07:40:00+08:00,2015-06-28 07:40:00+08:00
4,財經/要聞,《中國時報》\n●樂園變煉獄 派對驚爆 逾300人遭火紋身\n八仙樂園昨晚舉辦「COLOR ...,"Sun, 28 Jun 2015 07:38:17 +0800",6月28日各報頭版要聞,《 中國時報 》 ● 樂園 變 煉獄 派對 驚爆 逾 300 人 遭火 紋身 八仙 ...,6 月 28 日 各報 頭版 要聞,2015-06-28 07:38:17+08:00,2015-06-28 07:40:00+08:00
5,國際/政治,基改食品專題（中央社記者曾依璇巴黎28日專電）蘇菲（Sophie）活脫脫就是刻板印象中的巴黎...,"Sun, 28 Jun 2015 07:29:00 +0800",基改不算真食物 法國人知情就不買,基改 食品 專題 （ 中央社 記者 曾 依璇 巴黎 28 日 專電 ） 蘇菲 （ Sophi...,基改 不算 真 食物 法國人 知情 就 不 買,2015-06-28 07:29:00+08:00,2015-06-28 07:38:17+08:00
6,財經/要聞,傷者不斷被送進醫院，連計程車都加入，載送病患行列。誇張的是，有人是直接坐在救生圈上送醫，年輕...,"Sun, 28 Jun 2015 07:28:20 +0800",八仙樂園舞台大火 逾400人輕重傷,傷者 不斷 被 送進 醫院 ， 連 計程車 都 加入 ， 載送 病患 行列 。 誇張 的 是...,八仙 樂園 舞台 大火 逾 400 人 輕重傷,2015-06-28 07:28:20+08:00,2015-06-28 07:29:00+08:00
7,運動,（中央社加拿大艾德蒙頓27日綜合外電報導）衛冕的日本隊今天在加拿大艾德蒙頓舉行的女子世界盃足...,"Sun, 28 Jun 2015 07:28:00 +0800",女子世界盃足球賽 日擊敗澳洲晉4強,（ 中央社 加拿大 艾德蒙頓 27 日 綜合 外電 報導 ） 衛冕 的 日本隊 今天 在 加...,女子 世界盃 足球賽 日 擊敗 澳洲 晉 4 強,2015-06-28 07:28:00+08:00,2015-06-28 07:28:20+08:00
8,社會/生活,旺旺好運來！\n宜：納采、訂盟、冠笄、祭祀、祈福、齋醮、出行、修造、動土、移徙、入宅、安香、...,"Sun, 28 Jun 2015 07:12:37 +0800",《農民曆小幫手》國曆六月二十八日,旺旺 好運 來 ！ 宜 ： 納采 、 訂盟 、 冠笄 、 祭祀 、 祈福 、 齋醮 、 出行...,《 農民曆 小 幫手 》 國曆 六月 二十八日,2015-06-28 07:12:37+08:00,2015-06-28 07:28:00+08:00
9,休閒玩樂,二姐江蕙年初宣布封麥後，今年金曲獎也將最高榮耀「特別貢獻獎」頒發給她。江蕙在周杰倫的介紹下登...,"Sun, 28 Jun 2015 07:10:00 +0800",江蕙得「特別貢獻獎」 感恩金曲肯定她,二姐 江蕙 年初 宣布 封麥 後 ， 今年 金曲獎 也將 最高 榮耀 「 特別 貢獻獎 」 ...,江蕙 得 「 特別 貢獻獎 」 感恩 金曲 肯定 她,2015-06-28 07:10:00+08:00,2015-06-28 07:12:37+08:00
